In [ ]:
import ephemerista

ephemerista.init(eop_path="../../tests/resources/finals2000A.all.csv", spk_path="../../tests/resources/de440s.bsp")

In [ ]:
import random

import deap.base
import deap.creator
import deap.tools
import geojson_pydantic
import numpy as np
from deap import algorithms, tools

from ephemerista.analysis.coverage import Coverage
from ephemerista.constellation.design import Constellation, WalkerStar
from ephemerista.scenarios import Scenario
from ephemerista.time import Time, TimeDelta

In [ ]:
with open("../single_aoi.geojson") as f:
    aoi = geojson_pydantic.FeatureCollection.model_validate_json(f.read())

start_time = Time.from_iso("TDB", "2016-05-30T12:00:00")

scenario = Scenario(
    name="Coverage analysis with constellation",
    start_time=start_time,
    end_time=start_time + TimeDelta.from_days(5),
    areas_of_interest=aoi.features,
    discretization_resolution=10,
    time_step=600,
)

time = Time.from_iso("TDB", "2016-05-30T12:00:00")

PENALTY = 1e3


def evaluate_constellation(individual):
    nsats, nplanes = individual

    constellation = Constellation(
        model=WalkerStar(
            time=time,
            nsats=nsats,
            nplanes=nplanes,
            semi_major_axis=7000,
            inclination=45,
            eccentricity=0.0,
            periapsis_argument=90,
        )
    )

    scenario.constellations = [constellation]
    cov = Coverage(scenario=scenario)
    results = cov.analyze()
    mean_cov = np.array(results.coverage_percent).mean()
    return nsats, mean_cov


LOWER_BOUND_SATS = 5
UPPER_BOUND_SATS = 100
LOWER_BOUND_PLANES = 1
UPPER_BOUND_PLANES = 20

# Minimize number of satellites while maximizing coverage while favoring the latter
deap.creator.create("FitnessMulti", deap.base.Fitness, weights=(-0.5, 1.0))
deap.creator.create("Individual", list, fitness=deap.creator.FitnessMulti)  # type: ignore


def init_individual():
    """Randomly generate an individual ensuring integer values."""
    num_sats = random.randint(LOWER_BOUND_SATS, UPPER_BOUND_SATS)  # noqa: S311
    num_planes = random.randint(LOWER_BOUND_PLANES, UPPER_BOUND_PLANES)  # noqa: S311

    num_sats = max(num_planes, (num_sats // num_planes) * num_planes)

    return deap.creator.Individual([num_sats, num_planes])  # type: ignore


def mutate(individual, mu=0, sigma=5, indpb=0.2):
    num_sats, num_planes = individual

    # Apply Gaussian mutation
    if random.random() < indpb:  # noqa: S311
        num_sats += int(random.gauss(mu, sigma))
    if random.random() < indpb:  # noqa: S311
        num_planes += int(random.gauss(mu, sigma))

    # Enforce bounds and ensure integers
    num_sats = max(LOWER_BOUND_SATS, min(UPPER_BOUND_SATS, round(num_sats)))
    num_planes = max(LOWER_BOUND_PLANES, min(UPPER_BOUND_PLANES, round(num_planes)))

    num_sats = max(num_planes, (num_sats // num_planes) * num_planes)

    individual[:] = [num_sats, num_planes]
    return (individual,)


def crossover(ind1, ind2, indpb=0.5):
    for i in range(len(ind1)):
        if random.random() < indpb:  # Each gene has probability `indpb` of swapping  # noqa: S311
            ind1[i], ind2[i] = ind2[i], ind1[i]  # Swap values

    # Ensure num_sats remains divisible by num_planes
    for ind in [ind1, ind2]:
        num_sats, num_planes = ind
        if num_planes > 0:
            num_sats = max(num_planes, (num_sats // num_planes) * num_planes)
        ind[:] = [num_sats, num_planes]  # Assign values back

    return ind1, ind2


toolbox = deap.base.Toolbox()
toolbox.register("individual", init_individual)
toolbox.register("population", deap.tools.initRepeat, list, toolbox.individual)  # type: ignore
toolbox.register("evaluate", evaluate_constellation)
toolbox.register("mate", crossover, indpb=0.5)  # Blend crossover
toolbox.register("mutate", mutate, mu=0, sigma=5, indpb=0.2)  # Gaussian mutation
toolbox.register("select", deap.tools.selTournament, tournsize=2)

stats = tools.Statistics(key=lambda ind: ind.fitness.values)
stats.register("avg", np.mean, axis=0)
stats.register("std", np.std, axis=0)
stats.register("min", np.min, axis=0)
stats.register("max", np.max, axis=0)

hof = deap.tools.HallOfFame(1)


def optimize_constellation(pop_size=20, generations=50):
    pop = toolbox.population(n=pop_size)  # type: ignore

    pop = algorithms.eaSimple(
        pop, toolbox, cxpb=0.5, mutpb=0.1, ngen=generations, stats=stats, halloffame=hof, verbose=True
    )

    best_individual = deap.tools.selBest(pop, k=1)[0]
    best_sats, best_planes = best_individual
    best_cost = evaluate_constellation(best_individual)[0]

    print(f"Optimal Constellation: {best_sats} satellites, {best_planes} planes, Cost: {best_cost}")  # noqa: T201

In [ ]:
%timeit evaluate_constellation(init_individual())

In [ ]:
optimize_constellation()